In [ ]:
#import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [ ]:
!gdown 1-2dSpLswZeEcf9-wA1jovtSL9J23Z_JV
!gdown 1-4iRxubHVXR2656iPtpY4HQwaf1hVFjp

Downloading...
From: https://drive.google.com/uc?id=1-2dSpLswZeEcf9-wA1jovtSL9J23Z_JV
To: /content/df_recipes_filtered.csv
100% 3.39M/3.39M [00:00<00:00, 196MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4iRxubHVXR2656iPtpY4HQwaf1hVFjp
To: /content/df_reviews_filtered.csv
100% 989k/989k [00:00<00:00, 91.1MB/s]


In [ ]:
df_reviews = pd.read_csv('/content/df_reviews_filtered.csv', index_col=False)
df_recipes = pd.read_csv('/content/df_recipes_filtered.csv', index_col=False)

# Preprocessing

In [ ]:
df_recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,1999-09-17T04:47:00Z,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",...,31.9,405.8,557.2,29.1,3.1,5.0,45.3,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co..."
1,62,"Black Bean, Corn, and Tomato Salad",1570,Dave Miner,PT15M,PT10M,PT25M,1999-08-19T05:12:00Z,"This is easy, delicious, colorful, delicious, ...",character(0),...,2.3,0.0,20.0,55.8,16.6,4.3,17.1,2.0,NaN,"c(""In a bowl whisk together lemon juice, oil, ..."
2,67,Bourbon Pecan Pound Cake,1573,benluc,PT1H,PT1H,PT2H,1999-09-01T09:01:00Z,Make and share this Bourbon Pecan Pound Cake r...,character(0),...,13.6,143.6,441.9,91.2,1.9,62.6,7.8,12.0,NaN,"c(""Combine butter and sugar in bowl of electri..."
3,72,Brownie Pudding,193853,MirandaLee,PT35M,PT40M,PT1H15M,1999-08-20T09:50:00Z,Make and share this Brownie Pudding recipe fro...,character(0),...,3.6,7.8,160.2,34.0,2.1,25.3,2.3,10.0,NaN,"c(""Preheat oven to 350 degrees."", ""Combine flo..."
4,83,Bubble and Squeak,1544,tranch,PT2H27M,PT30M,PT2H57M,1999-08-28T02:59:00Z,Make and share this Bubble and Squeak recipe f...,character(0),...,11.5,54.9,286.1,56.9,7.6,4.9,11.1,4.0,NaN,"c(""Wash potatoes under cold running water then..."


In [ ]:
df_recipes.iloc[0]

RecipeId                                                                     44
Name                                                     Warm Chicken A La King
AuthorId                                                                   1596
AuthorName                                                        Joan Edington
CookTime                                                                   PT3M
PrepTime                                                                  PT35M
TotalTime                                                                 PT38M
DatePublished                                              1999-09-17T04:47:00Z
Description                   I copied this one out of a friend's book so ma...
Images                        "https://img.sndimg.com/food/image/upload/w_55...
RecipeCategory                                                          Chicken
Keywords                                      c("Poultry", "Meat", "< 60 Mins")
RecipeIngredientQuantities    c("12", "2

In [ ]:
df_reviews.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,0,0,gayg msft,5.0,better than any you can get at a restaurant!,2000-01-25 21:44:00+00:00,2000-01-25T21:44:00Z
1,7,1,1,Bill Hilbrich,4.0,"I cut back on the mayo, and made up the differ...",2001-10-17 16:49:59+00:00,2001-10-17T16:49:59Z
2,9,2,2,Gay Gilmore ckpt,2.0,i think i did something wrong because i could ...,2000-02-25 09:00:00+00:00,2000-02-25T09:00:00Z
3,13,3,3,Malarkey Test,5.0,easily the best i have ever had. juicy flavor...,2000-03-13 21:15:00+00:00,2000-03-13T21:15:00Z
4,14,4,4,Tony Small,5.0,An excellent dish.,2000-03-28 12:51:00+00:00,2000-03-28T12:51:00Z


In [ ]:
df_reviews.iloc[0]

ReviewId                                                    2
RecipeId                                                    0
AuthorId                                                    0
AuthorName                                          gayg msft
Rating                                                    5.0
Review           better than any you can get at a restaurant!
DateSubmitted                       2000-01-25 21:44:00+00:00
DateModified                             2000-01-25T21:44:00Z
Name: 0, dtype: object

###Drop Columns, reduce rows of data, and join reviews

In [ ]:
recipe_cols = ['RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime', 'Description', 'Images',\
              'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', \
              'AggregatedRating', 'Calories', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']
review_cols = ['ReviewId', 'RecipeId', 'Rating', 'Review']

In [ ]:
# limit recipe entries
rp_df = df_recipes[recipe_cols]
rv_df = df_reviews[review_cols]
# extract rows with reviews
rp_df = pd.DataFrame.merge(rp_df, rv_df.RecipeId, on='RecipeId').drop_duplicates('RecipeId')

In [ ]:
# Checking null values
rp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 398
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    233 non-null    int64  
 1   Name                        233 non-null    object 
 2   CookTime                    151 non-null    object 
 3   PrepTime                    233 non-null    object 
 4   TotalTime                   233 non-null    object 
 5   Description                 233 non-null    object 
 6   Images                      233 non-null    object 
 7   RecipeCategory              233 non-null    object 
 8   Keywords                    230 non-null    object 
 9   RecipeIngredientQuantities  233 non-null    object 
 10  RecipeIngredientParts       233 non-null    object 
 11  AggregatedRating            231 non-null    float64
 12  Calories                    233 non-null    float64
 13  RecipeServings              165 non

In [ ]:
recipe_col_subset = ['RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime', 'Description', 'Images',\
              'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', \
              'AggregatedRating', 'Calories', 'RecipeInstructions']

rp_df = rp_df.dropna(subset=recipe_col_subset)
rp_df = rp_df[rp_df.Images != 'character(0)']
rp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 398
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    70 non-null     int64  
 1   Name                        70 non-null     object 
 2   CookTime                    70 non-null     object 
 3   PrepTime                    70 non-null     object 
 4   TotalTime                   70 non-null     object 
 5   Description                 70 non-null     object 
 6   Images                      70 non-null     object 
 7   RecipeCategory              70 non-null     object 
 8   Keywords                    70 non-null     object 
 9   RecipeIngredientQuantities  70 non-null     object 
 10  RecipeIngredientParts       70 non-null     object 
 11  AggregatedRating            70 non-null     float64
 12  Calories                    70 non-null     float64
 13  RecipeServings              51 non-n

In [ ]:
# reduce rows
rp_df = rp_df[:10000]
rv_df = pd.DataFrame.merge(rv_df, rp_df.RecipeId, how='inner')

In [ ]:
%%time
# This take about 35s with 10000 rows
# RecipeId as a key, list of dict as a value
rc_rv_dict = dict()
for name, df in rv_df.groupby('RecipeId'):
    # drop unnecessary column, set index and transpose (to create dict)
    df = df.drop('RecipeId', axis=1).set_index('ReviewId').T
    dict1 = df.to_dict('dict')
    rc_rv_dict[name] = dict1

CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 153 ms


In [ ]:
# add a new column
rp_df['reviews_in_dict'] = rp_df.RecipeId.apply(lambda x: rc_rv_dict[x])

In [ ]:
#Dropping unneccesary columns
modify_col_list = ['Images', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'RecipeInstructions']
# rp_df = rp_df.apply(lambda x: map_for_series(x) if x.name in modify_col_list else x)

In [ ]:
import re

def map_str_to_list(string):
    #pattern = re.compile(r'\"(.+)\"')
    pattern = re.compile(r'\"([^"]+)\"')
    return pattern.findall(string)

def map_for_series(series: pd.Series):
    return series.apply(lambda i: map_str_to_list(i))

In [ ]:
rp_df.reviews_in_dict.iat[4]

{262: {'Rating': 0.0, 'Review': 'great recipe its delicious'}}

In [ ]:
rp_df.reset_index().to_csv('recipe_10000.csv')

###Give Recommendations

In [ ]:
def score_recipes(user_input, df, best_num):
    '''
    user_input: list of strings
    df: our list of recipes
    best_num: number of best matching result to return
    '''
    df = df.copy()

    def score(ingredient_list):
        score = 0
        for w in user_input:
            if w in ingredient_list:
                score += 1
        return score

    df['score'] = df['RecipeIngredientParts'].apply(lambda x: score(x))
    df = df.sort_values(by='score', ascending=False).iloc[:best_num]
    return df

In [ ]:
#Giving reccomendations based on user inputs
user_input = ['potato', 'carrots', 'pork', 'chestnuts', 'butter', 'salmon']
score_recipes(user_input, rp_df, 10)

,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,Calories,RecipeServings,RecipeYield,RecipeInstructions,reviews_in_dict,score
71,267,Twice Baked Potatoes,PT1H45M,PT50M,PT2H35M,Make and share this Twice Baked Potatoes recip...,"c(""https://img.sndimg.com/food/image/upload/w_...",Potato,"c(""Vegetable"", ""Weeknight"", ""Oven"", ""< 4 Hours"")","c(""6"", ""1/2"", ""1/4"", ""1"", ""1/2"", ""1/4"", ""1"", ""...","c(""baking potatoes"", ""milk"", ""butter"", ""egg"", ...",5.0,186.0,NaN,8 halves,"c(""Heat oven to 400 degrees Fahrenheit."", ""Scr...","{480: {'Rating': 4.0, 'Review': 'This recipe i...",2
0,44,Warm Chicken A La King,PT3M,PT35M,PT38M,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""Poultry"", ""Meat"", ""< 60 Mins"")","c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,895.5,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co...","{90: {'Rating': 5.0, 'Review': 'quite easy to ...",1
328,1293,Seven Layer Magic Bars,PT25M,PT10M,PT35M,Make and share this Seven Layer Magic Bars rec...,"c(""https://img.sndimg.com/food/image/upload/w_...",Bar Cookie,"c(""Dessert"", ""Cookie & Brownie"", ""< 60 Mins"", ...","c(""1/2"", ""1 1/2"", ""1"", ""1"", ""1"", ""1 1/3"", ""1"")","c(""butter"", ""margarine"", ""graham cracker crumb...",5.0,157.6,36.0,24-35 bars,"c(""Preheat oven to 350 (325 for glass dish). I...","{3984: {'Rating': 5.0, 'Review': 'A lovely rec...",1
221,607,Famous Barr's French Onion Soup,PT3H45M,PT40M,PT4H25M,One of St. Louis's favorite soup recipes was r...,"c(""https://img.sndimg.com/food/image/upload/w_...",Cheese,"c(""Onions"", ""Vegetable"", ""European"", ""Low Prot...","c(""5"", ""1/2"", ""1 1/2"", ""2"", ""1"", ""7"", ""1"", ""3/...","c(""onions"", ""butter"", ""black pepper"", ""paprika...",5.0,145.8,16.0,4 quarts,"c(""Peel onions and slice 1/8 inch thick, prefe...","{1303: {'Rating': 4.0, 'Review': 'This sounds ...",1
225,617,Easy Refrigerator Lemon Cookies,PT10M,PT45M,PT55M,This is my adopted recipe as of Feb 2005. I ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Dessert,"c(""Cookie & Brownie"", ""< 60 Mins"", ""Oven"", ""Re...","c(""1"", ""1"", ""1"", ""2 1/3"", ""1/2"", ""1/4"")","c(""lemon, zest of"", ""granulated sugar"", ""butte...",4.5,59.3,NaN,60 cookies,"c(""Process the lemon zest and sugar in a food ...","{1332: {'Rating': 0.0, 'Review': 'I only got o...",1
17,102,Cheesy Scalloped Potato Side Dish,PT1H30M,PT25M,PT1H55M,--Adopted Recipe--\r\nThis is a simple scallop...,"c(""https://img.sndimg.com/food/image/upload/w_...",Potato,"c(""Cheese"", ""Vegetable"", ""Healthy"", ""Weeknight...","c(""5"", ""1/4"", ""1 1/2"", ""1 1/2"", ""1/2"", NA, NA)","c(""potatoes"", ""flour"", ""half-and-half"", ""chedd...",4.0,332.0,NaN,NaN,"c(""Layer potatoes, flour, milk, and salt and p...","{179: {'Rating': 5.0, 'Review': 'Fantastic Yo...",1
240,657,Dinner Rolls,PT2H35M,PT2H10M,PT4H45M,Make and share this Dinner Rolls recipe from F...,"c(""https://img.sndimg.com/food/image/upload/w_...",Yeast Breads,"c(""Breads"", ""Healthy"", ""Weeknight"", ""Oven"")","c(""8"", ""2"", ""2"", ""2 1/2"", ""2"", ""2"", ""18"")","c(""milk"", ""unsalted butter"", ""sugar"", ""dried y...",5.0,175.2,NaN,16 2ounce rolls,"c(""For the glaze:- 1 egg yolk 1 Tbsp water Put...","{1430: {'Rating': 5.0, 'Review': 'A sweet/sour...",1
246,665,Crock Pot Potatoes,PT6H,PT10M,PT6H10M,Make and share this Crock Pot Potatoes recipe ...,"""https://img.sndimg.com/food/image/upload/w_55...",Potato,"c(""Vegetable"", ""Low Protein"", ""Free Of..."", ""W...","c(""2"", ""2"", ""1"", ""8"", ""1"")","c(""garlic cloves"", ""salt"", ""potatoes"", ""cream ...",4.0,536.9,NaN,NaN,"c(""Lightly grease Crock-Pot. In small bowl, c...","{1457: {'Rating': 0.0, 'Review': 'A very filli...",1
264,810,Irresistible Peanut 

In [ ]:
# user_input = ['great']
# score_recipes(user_input, rv_df, 4)

In [ ]:
user_input = ['rice']
score_recipes(user_input, rp_df, 5)

,RecipeId,Name,CookTime,PrepTime,TotalTime,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,Calories,RecipeServings,RecipeYield,RecipeInstructions,reviews_in_dict,score
0,44,Warm Chicken A La King,PT3M,PT35M,PT38M,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""Poultry"", ""Meat"", ""< 60 Mins"")","c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,895.5,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co...","{90: {'Rating': 5.0, 'Review': 'quite easy to ...",0
266,834,Best Hamburger Diane,PT1M,PT0S,PT1M,We made this the other night and loved it! I ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Meat,"c(""Very Low Carbs"", ""High Protein"", ""High In.....","c(""1"", ""3/4"", ""1/8"", ""2"", ""1"", ""1"", ""1/2"")","c(""salt"", ""pepper"", ""butter"", ""prepared mustar...",4.5,458.6,3.0,NaN,"c(""Mix the ground round, salt and pepper toget...","{1947: {'Rating': 5.0, 'Review': 'Diana, Thes...",0
318,1168,Swiss Chocolate Squares & Icing,PT25M,PT1H,PT1H25M,Make and share this Swiss Chocolate Squares & ...,"""https://img.sndimg.com/food/image/upload/w_55...",Bar Cookie,"c(""Dessert"", ""Cookie & Brownie"", ""Swiss"", ""Eur...","c(""1"", ""1/2"", ""1 1/2"", ""2"", ""2"", ""2"", ""1/2"", ""...","c(""water"", ""butter"", ""margarine"", ""flour"", ""su...",5.0,293.7,24.0,NaN,"c(""Preheat oven to 375 degrees."", ""Combine wat...","{3658: {'Rating': 4.0, 'Review': 'This recipe ...",0
309,1136,Spoon Bread Chili Pie,PT35M,PT15M,PT50M,Make and share this Spoon Bread Chili Pie reci...,"""https://img.sndimg.com/food/image/upload/w_55...",Savory Pies,"c(""Meat"", ""Free Of..."", ""< 60 Mins"", ""Oven"")","c(""1/2"", ""1"", ""1/4"", ""1"", ""1"", ""1 1/4"", ""1 1/2...","c(""beef"", ""onion"", ""green pepper"", ""garlic clo...",4.5,529.3,6.0,NaN,"c(""Cook ground beef, onion, green pepper and g...","{3544: {'Rating': 5.0, 'Review': 'A lovely Bre...",0
303,1106,Soft Zucchini Spice Cookies,PT8M,PT15M,PT23M,Make and share this Soft Zucchini Spice Cookie...,"""https://img.sndimg.com/food/image/upload/w_55...",Drop Cookies,"c(""Dessert"", ""Cookie & Brownie"", ""Vegetable"", ...","c(""1/2"", ""1"", ""1"", ""1 3/4"", ""2"", ""3/4"", ""1/2"",...","c(""butter"", ""margarine"", ""brown sugar"", ""egg"",...",4.5,66.7,NaN,48 cookies,"c(""In a mixing bowl, cream the butter and brow...","{3441: {'Rating': 4.0, 'Review': 'Too much gar...",0


In [ ]:
# Step 3: Using TfidfVectorizer to convert text data into TF-IDF features
tfidf = TfidfVectorizer(stop_words='english')
df_recipes['Description'] = df_recipes['Description'].fillna('')  # replacing NaN with empty string
tfidf_matrix = tfidf.fit_transform(df_recipes['Description'])

# Step 4: Calculating similarity between recipes using linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Step 5: Write a function to recommend recipes based on similarity scores
def recommend_recipes(recipe_id, num_recommendations=5):
    # Get the index of the recipe that matches the title
    idx = recipe_id

    # Get the pairwsie similarity scores of all recipes with that recipe
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the recipes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top-N most similar recipes
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the recipe indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top-N most similar recipes
    return df_recipes['Name'].iloc[recipe_indices]

# Example usage with RecipeId = 44 (first entry in our dataframe)
recommended_recipes = recommend_recipes(44, 5)
recommended_recipes


551      Crusty Coconut Custard
42              Thai Beef Salad
589                Coconut Rice
59     Sweet Potato Custard Pie
37           Thai Chicken Curry
Name: Name, dtype: object

In [ ]:
def recommend_recipes(recipe_id, num_recommendations=5):
    # Get the index of the recipe that matches the title
    idx = recipe_id

    # Get the pairwsie similarity scores of all recipes with that recipe
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the recipes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top-N most similar recipes
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the recipe indices
    recipe_indices = [i[0] for i in sim_scores]

    # Return the top-N most similar recipes along with their information
    return df_recipes.iloc[recipe_indices]

# Example usage with RecipeId = 44 (first entry in our dataframe)
recommended_recipes = recommend_recipes(44, 5)
print(recommended_recipes)


     RecipeId                      Name  AuthorId          AuthorName  \
551      4539    Crusty Coconut Custard      1762             lkadlec   
42        285           Thai Beef Salad      1601             Gilcat2   
589      4785              Coconut Rice      1556     Strawberry Girl   
59        380  Sweet Potato Custard Pie      1535  Marg CaymanDesigns   
37        250        Thai Chicken Curry      1601             Gilcat2   

    CookTime PrepTime TotalTime         DatePublished  \
551    PT30M    PT30M      PT1H  1999-11-15T05:39:00Z   
42      PT1H    PT30M   PT1H30M  1999-09-04T20:47:00Z   
589    PT20M     PT5M     PT25M  1999-12-01T20:03:00Z   
59     PT50M    PT15M    PT1H5M  1999-09-03T14:34:00Z   
37     PT23M    PT10M     PT33M  1999-09-04T20:47:00Z   

                                           Description  \
551  Make and share this Crusty Coconut Custard rec...   
42   Make and share this Thai Beef Salad recipe fro...   
589  Make and share this Coconut Rice recipe

# RATING SECTION

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!gdown 1-2dSpLswZeEcf9-wA1jovtSL9J23Z_JV
!gdown 1-4iRxubHVXR2656iPtpY4HQwaf1hVFjp

Downloading...
From: https://drive.google.com/uc?id=1-2dSpLswZeEcf9-wA1jovtSL9J23Z_JV
To: /content/df_recipes_filtered.csv
100% 3.39M/3.39M [00:00<00:00, 183MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4iRxubHVXR2656iPtpY4HQwaf1hVFjp
To: /content/df_reviews_filtered.csv
100% 989k/989k [00:00<00:00, 76.5MB/s]


In [ ]:
df_reviews = pd.read_csv('/content/df_reviews_filtered.csv', index_col=False)
df_recipes = pd.read_csv('/content/df_recipes_filtered.csv', index_col=False)

In [ ]:
df_reviews

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,0,0,gayg msft,5.0,better than any you can get at a restaurant!,2000-01-25 21:44:00+00:00,2000-01-25T21:44:00Z
1,7,1,1,Bill Hilbrich,4.0,"I cut back on the mayo, and made up the differ...",2001-10-17 16:49:59+00:00,2001-10-17T16:49:59Z
2,9,2,2,Gay Gilmore ckpt,2.0,i think i did something wrong because i could ...,2000-02-25 09:00:00+00:00,2000-02-25T09:00:00Z
3,13,3,3,Malarkey Test,5.0,easily the best i have ever had. juicy flavor...,2000-03-13 21:15:00+00:00,2000-03-13T21:15:00Z
4,14,4,4,Tony Small,5.0,An excellent dish.,2000-03-28 12:51:00+00:00,2000-03-28T12:51:00Z
...,...,...,...,...,...,...,...,...
4003,1429155,696,1368,Carol,5.0,WOW! These cookies are without a doubt the bes...,2001-11-22 10:48:19+00:00,2001-11-22T10:48:19Z
4004,1429159,696,1166,Ellen Brody,5.0,These were really easy to make and are very so...,2001-12-08 05:25:34+00:00,2001-12-08T05:25:34Z
4005,1429163,696,1451,selvi nadarajah,5.0,"thanks Dawn,\r\nthis was superb.I made it for ...",2001-11-14 09:37:41+00:00,2001-11-14T09:37:41Z
4006,1429170,696,1589,Usako Yamakawa,2.0,"Maybe I did something wrong, or maybe I was ex...",2001-12-22 16:17:03+00:00,2001-12-22T16:17:03Z


In [ ]:
df_recipes

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,44,Warm Chicken A La King,1596,Joan Edington,PT3M,PT35M,PT38M,1999-09-17T04:47:00Z,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",...,31.9,405.8,557.2,29.1,3.1,5.0,45.3,2.0,NaN,"c(""Melt 1 1/2 ozs butter, add the flour and co..."
1,62,"Black Bean, Corn, and Tomato Salad",1570,Dave Miner,PT15M,PT10M,PT25M,1999-08-19T05:12:00Z,"This is easy, delicious, colorful, delicious, ...",character(0),...,2.3,0.0,20.0,55.8,16.6,4.3,17.1,2.0,NaN,"c(""In a bowl whisk together lemon juice, oil, ..."
2,67,Bourbon Pecan Pound Cake,1573,benluc,PT1H,PT1H,PT2H,1999-09-01T09:01:00Z,Make and share this Bourbon Pecan Pound Cake r...,character(0),...,13.6,143.6,441.9,91.2,1.9,62.6,7.8,12.0,NaN,"c(""Combine butter and sugar in bowl of electri..."
3,72,Brownie Pudding,193853,MirandaLee,PT35M,PT40M,PT1H15M,1999-08-20T09:50:00Z,Make and share this Brownie Pudding recipe fro...,character(0),...,3.6,7.8,160.2,34.0,2.1,25.3,2.3,10.0,NaN,"c(""Preheat oven to 350 degrees."", ""Combine flo..."
4,83,Bubble and Squeak,1544,tranch,PT2H27M,PT30M,PT2H57M,1999-08-28T02:59:00Z,Make and share this Bubble and Squeak recipe f...,character(0),...,11.5,54.9,286.1,56.9,7.6,4.9,11.1,4.0,NaN,"c(""Wash potatoes under cold running water then..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,70137,Mexicali Spoon Bread Casserole,51579,SilentCricket,PT40M,PT30M,PT1H10M,2003-09-02T20:01:00Z,In the mood for something Mexican and/or cornb...,character(0),...,11.8,171.0,1392.1,34.2,4.3,6.4,32.8,NaN,1 casserole,"c(""Brown beef, onions, green pepper, and garli..."
2513,109055,Mediterranean Penne,148316,- Carla -,PT25M,PT20M,PT45M,2005-01-21T20:03:00Z,"I don't know how this recipe came to be, I've ...","c(""https://img.sndimg.com/food/image/upload/w_...",...,12.6,62.4,1381.2,50.6,8.9,5.1,15.1,6.0,NaN,"c(""In a large pot, cook penne according to pac..."
2514,161382,Island Mango Bread,217657,startnover,NaN,PT10M,PT10M,2006-03-24T15:55:00Z,Lots of flavors come together to make a yummy ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,41.0,431.8,4578.9,455.7,12.8,259.6,44.6,NaN,1 loaf,"c(""Mix sour cream and margarine together and t..."
2515,161399,Sweet Potato and Mango Bake,128473,Baby Kato,PT1H,PT30M,PT1H30M,2006-03-24T16:00:00Z,A nice side dish served with savoury chicken a...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,15.3,360.6,45.5,4.4,27.6,2.8,4.0,NaN,"c(""Preheat oven to 350°F."", ""Grease pan lightl..."


In [ ]:
N = 10
top_N_frequent_recipe_ids = df_reviews['RecipeId'].value_counts().head(N)
top_N_frequent_recipe_ids

858     29
1423    24
376     16
411     15
105     15
282     15
714     14
15      13
307     12
696     11
Name: RecipeId, dtype: int64

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163692 sha256=f74f21b3b2dd09e6b2e76c37adba880d5a5262eacfe6b64cb5a231ac7de4832a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Bseline, KNN, and SVD

In [ ]:
from surprise import SVD
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Assuming df_reviews contains the 'AuthorId', 'RecipeId', 'Rating' columns

# Define the rating scale (if it's not specified, it defaults to (1, 5))
reader = Reader(rating_scale=(1, 5))

# Load the dataset from the CSV file
data = Dataset.load_from_df(df_reviews[['AuthorId', 'RecipeId', 'Rating']], reader)

baseline_only = BaselineOnly()
cross_validate(baseline_only, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# KNN Basic
knn_basic = KNNBasic()
cross_validate(knn_basic, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# SVD
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5362  1.4928  1.5244  1.5398  1.4095  1.5005  0.0484  
MAE (testset)     1.0895  1.0670  1.0990  1.0867  1.0367  1.0758  0.0221  
Fit time          0.02    0.01    0.02    0.02    0.02    0.02    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic

{'test_rmse': array([1.49673768, 1.41759774, 1.53120982, 1.46466301, 1.5580684 ]),
 'test_mae': array([1.06463448, 1.0334654 , 1.09985302, 1.03625567, 1.11143243]),
 'fit_time': (0.07535529136657715,
  0.0740671157836914,
  0.07337570190429688,
  0.07886648178100586,
  0.07695698738098145),
 'test_time': (0.007561922073364258,
  0.007345914840698242,
  0.004042387008666992,
  0.0041866302490234375,
  0.008322477340698242)}

# HYRBID

In [ ]:
class HybridModel:
    def __init__(self):
        self.baseline_only = BaselineOnly(bsl_options={'method': 'sgd', 'reg': 0.02, 'learning_rate': 0.005})
        self.svd = SVD(reg_all=0.02)
        self.knn_basic = KNNBasic(k=50)

    def fit(self, trainset):
        self.baseline_only.fit(trainset)
        self.svd.fit(trainset)
        self.knn_basic.fit(trainset)

    def predict(self, testset):
      predictions_baseline = self.baseline_only.test(testset)
      predictions_svd = self.svd.test(testset)

      # Weighted predictions for SVD and baseline
      weighted_predictions_svd = 0.8 * np.array([pred.est for pred in predictions_svd])
      weighted_predictions_baseline = 0.2 * np.array([pred.est for pred in predictions_baseline])

      # Combine SVD and baseline predictions
      final_weighted_predictions = weighted_predictions_svd + weighted_predictions_baseline

      predictions_knn = self.knn_basic.test(testset)

      most_popular = df_reviews.groupby('RecipeId')['Rating'].count().sort_values(ascending=False).index
      most_popular_predictions = [self.baseline_only.predict(uid='dummy', iid=recipe_id).est for recipe_id in most_popular[:len(testset)]]

      # Add predictions from KNN and most popular
      final_weighted_predictions += np.array([pred.est for pred in predictions_knn]) + np.array(most_popular_predictions)

      # Normalize the final predictions
      final_weighted_predictions /= 3.0

      # Format predictions as a list of Prediction objects
      predictions = [
          Prediction(uid=pred.uid, iid=pred.iid, r_ui=pred.r_ui, est=final_est, details={})
          for pred, final_est in zip(predictions_svd, final_weighted_predictions)
      ]

      return predictions


    def test(self, testset):
        return self.predict(testset)

# Define the rating scale (if it's not specified, it defaults to (1, 5))
reader = Reader(rating_scale=(1, 5))

# Load the dataset from the CSV file
data = Dataset.load_from_df(df_reviews[['AuthorId', 'RecipeId', 'Rating']], reader)

# Define the cross-validation iterator
kf = KFold(n_splits=5, shuffle=True)

# Create an instance of the HybridModel
hybrid_model_instance = HybridModel()

# Perform cross-validation
cv_results = cross_validate(hybrid_model_instance, data, measures=['RMSE', 'MSE', 'MAE'], cv=kf, verbose=True)

# Display the cross-validation results
print(f'Average RMSE: {np.mean(cv_results["test_rmse"])}')
print(f'Average MSE: {np.mean(cv_results["test_mse"])}')
print(f'Average MAE: {np.mean(cv_results["test_mae"])}')


Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm HybridModel on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5931  1.4380  1.5516  1.5186  1.5618  1.5326  0.0530  
MSE (testset)     2.5381  2.0677  2.4075  2.3060  2.4391  2.3517  0.1601  
MAE (testset)     1.1180  1.0577  1.1163  1.0987  1.1212  1.1024  0.0237  
Fit time          0.29    0.18    0.22    0.18    0.19    0.21    0.04    
Test time         0.03    0.03    0.03

# HYBDRID with Content Based

In [ ]:
from surprise import Dataset, Reader, BaselineOnly, KNNBasic, SVD
from surprise.model_selection import cross_validate, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from surprise import Prediction
from surprise import accuracy
from surprise.model_selection import train_test_split

class HybridModel:
    def __init__(self, df_recipes):
        self.baseline_only = BaselineOnly(bsl_options={'method': 'sgd', 'reg': 0.02, 'learning_rate': 0.005})
        self.svd = SVD(reg_all=0.02)
        self.knn_basic = KNNBasic(k=50)

        # Content-based variables
        self.tfidf = TfidfVectorizer(stop_words='english')
        df_recipes['Description'] = df_recipes['Description'].fillna('')
        self.tfidf_matrix = self.tfidf.fit_transform(df_recipes['Description'])
        self.cosine_sim = linear_kernel(self.tfidf_matrix, self.tfidf_matrix)

    def fit(self, df_reviews):
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(df_reviews[['AuthorId', 'RecipeId', 'Rating']], reader)

        # Perform a train-test split
        trainset, testset = train_test_split(data, test_size=0.2)

        # Fit the individual models
        self.baseline_only.fit(trainset)
        self.svd.fit(trainset)
        self.knn_basic.fit(trainset)

        # Save the testset for later use in the predict method
        self.testset = testset

    def predict(self, df_recipes):
        # Assuming df_recipes and testset are defined before this point
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(df_recipes[['AuthorId', 'RecipeId', 'Rating']], reader)
        testset = data.build_full_trainset().build_testset()

        predictions_svd = self.svd.test(testset)
        predictions_baseline = self.baseline_only.fit(testset)

        # Weighted predictions for SVD and baseline
        weighted_predictions_svd = 0.8 * np.array([pred.est for pred in predictions_svd])
        weighted_predictions_baseline = 0.2 * np.array([pred.est for pred in predictions_baseline])

        # Combine SVD and baseline predictions
        final_weighted_predictions = weighted_predictions_svd + weighted_predictions_baseline

        predictions_knn = self.knn_basic.test(testset)

        most_popular = df_reviews.groupby('RecipeId')['Rating'].count().sort_values(ascending=False).index
        most_popular_predictions = [self.baseline_only.predict(uid='dummy', iid=recipe_id).est for recipe_id in most_popular[:len(testset)]]

        # Add predictions from KNN and most popular
        final_weighted_predictions += np.array([pred.est for pred in predictions_knn]) + np.array(most_popular_predictions)

        # Normalize the final predictions
        final_weighted_predictions /= 3.0

        # Content-based recommendations
        content_recommendations = self.recommend_content(testset, df_recipes)

        # Combine collaborative and content-based predictions
        combined_predictions = final_weighted_predictions + content_recommendations

        # Format predictions as a list of Prediction objects
        predictions = [
            Prediction(uid=pred.uid, iid=pred.iid, r_ui=pred.r_ui, est=combined_est, details={})
            for pred, combined_est in zip(predictions_svd, combined_predictions)
        ]

        return predictions

    def recommend_content(self, testset, df_recipes, num_recommendations=5):
        content_recommendations = []

        for user_id, recipe_id, _ in testset:
            content_recipes = self.recommend_recipes(recipe_id, num_recommendations)
            # Format content-based recommendations as a list of tuples (user_id, recommended_recipe)
            content_recommendations.extend([(user_id, recipe) for recipe in content_recipes])

        return content_recommendations

    def recommend_recipes(self, recipe_id, num_recommendations=5):
        # Get the index of the recipe that matches the RecipeId
        idx = recipe_id

        # Get the pairwise similarity scores of all recipes with that recipe
        sim_scores = list(enumerate(self.cosine_sim[idx]))

        # Sort the recipes based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the top-N most similar recipes
        sim_scores = sim_scores[1:num_recommendations + 1]

        # Get the recipe indices
        recipe_indices = [i[0] for i in sim_scores]

        # Return the top-N most similar recipes
        return df_recipes['Name'].iloc[recipe_indices]

    def test(self, testset, df_recipes):
        return self.predict(testset, df_recipes)

# Create an instance of the HybridModel
hybrid_model_instance = HybridModel(df_recipes)

# Fit the model
hybrid_model_instance.fit(df_reviews)

# Perform cross-validation using the SVD algorithm
cv_results = cross_validate(hybrid_model_instance.svd, data, measures=['RMSE', 'MSE', 'MAE'], cv=kf, verbose=True)

# Display the cross-validation results
print(f'Average RMSE: {np.mean(cv_results["test_rmse"])}')
print(f'Average MSE: {np.mean(cv_results["test_mse"])}')
print(f'Average MAE: {np.mean(cv_results["test_mae"])}')

# Display the content recommendations
print("\nContent-based Recommendations:")
for i, (user_id, recipe_name) in enumerate(content_recommendations):
    if i < 10:
        print(f"User {user_id} - Recommended Recipe: {recipe_name}")
    else:
        break

# Display using recommend_recipes function
recommended_recipes = recommend_recipes(44, 5)
recommended_recipes

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5089  1.5947  1.5254  1.5118  1.5904  1.5462  0.0383  
MSE (testset)     2.2766  2.5431  2.3268  2.2857  2.5295  2.3923  0.1188  
MAE (testset)     1.0992  1.1401  1.0774  1.0709  1.1180  1.1011  0.0256  
Fit time          0.09    0.08    0.08    0.09    0.10    0.09    0.01    
Test time         0.00    0.00    0.00    0.00    0.01    0.00    0.00    
Average RMSE: 1.546242265172431
Average MSE: 2.392328523200638
Average MAE: 1.1011169814829223

Content-based Recommendations:
User 6303 - Recommended Recipe: Mrs. Fields Chocolate Chip Cookies
User 6303 - Recommended Recipe: Orange Cookies
User 6303 - Recommended Recipe: Orange Bread
User 6303 - Recommended Recipe: Quick and Easy No Bake Chocolate Cookies
User 6303 - Recommended Reci

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
551,4539,Crusty Coconut Custard,1762,lkadlec,PT30M,PT30M,PT1H,1999-11-15T05:39:00Z,Make and share this Crusty Coconut Custard rec...,character(0),...,10.1,81.5,39.7,42.2,2.6,38.9,4.0,8.0,NaN,"c(""Preheat oven to 350°F"", ""In a bowl, dump co..."
42,285,Thai Beef Salad,1601,Gilcat2,PT1H,PT30M,PT1H30M,1999-09-04T20:47:00Z,Make and share this Thai Beef Salad recipe fro...,character(0),...,38.9,112.4,862.8,26.5,12.7,11.1,18.2,4.0,NaN,"c(""Tear romaine lettuce, red leaf lettuce, and..."
589,4785,Coconut Rice,1556,Strawberry Girl,PT20M,PT5M,PT25M,1999-12-01T20:03:00Z,Make and share this Coconut Rice recipe from F...,character(0),...,21.4,7.6,36.5,33.5,0.9,1.2,4.9,4.0,NaN,"c(""In a large saucepan, melt butter over mediu..."
59,380,Sweet Potato Custard Pie,1535,Marg CaymanDesigns,PT50M,PT15M,PT1H5M,1999-09-03T14:34:00Z,Make and share this Sweet Potato Custard Pie r...,character(0),...,4.4,64.2,339.0,27.6,2.1,8.1,5.4,8.0,NaN,"c(""Combine eggs, sugar, salt, nutmeg, cinnamon..."
37,250,Thai Chicken Curry,1601,Gilcat2,PT23M,PT10M,PT33M,1999-09-04T20:47:00Z,Make and share this Thai Chicken Curry recipe ...,"""https://img.sndimg.com/food/image/upload/w_55...",...,23.7,85.1,1163.0,32.2,4.1,1.8,27.2,4.0,NaN,"c(""Combine cocnut milk, potatoes, nam pla, cur..."
